In [1]:
import torch.nn as nn
import torch.nn.functional as F

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)
    
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [2]:
import h5py
import io
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, csv_file, img_hdf5_file, transform=None, is_test=False):
        self.data = pd.read_csv(csv_file)
        self.img_hdf5_file = img_hdf5_file
        self.transform = transform
        self.is_test = is_test
        
        # Process image IDs
        self.image_ids = self.data['isic_id'].values
        
        # Process structured data
        if 'target' in self.data.columns:
            self.structured_data = self.data.drop(columns=['isic_id', 'target'])
        else:
            self.structured_data = self.data.drop(columns=['isic_id'])
        
        for col in self.structured_data.columns:
            if self.structured_data[col].dtype == 'object':
                self.structured_data[col] = self.structured_data[col].astype(str).str.extract('(\d+)').astype(float)
        
        self.structured_data = self.structured_data.fillna(0).values.astype(np.float32)
        
        if not is_test:
            self.targets = self.data['target'].values.astype(np.float32)
        else:
            self.targets = None

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        with h5py.File(self.img_hdf5_file, 'r') as hdf5_file:
            image_key = self.image_ids[idx]
            image_data = hdf5_file[image_key][()]
            
        image = Image.open(io.BytesIO(image_data))
        if self.transform:
            image = self.transform(image)
        
        structured_data = torch.tensor(self.structured_data[idx], dtype=torch.float32)
        if self.targets is not None:
            target = torch.tensor(self.targets[idx], dtype=torch.float32)
            return image, structured_data, target
        else:
            return image, structured_data, self.image_ids[idx]


In [3]:
def extract_features(model, dataloader, device):
    model.eval()
    all_features = []
    all_targets = []

    with torch.no_grad():
        for images, _, targets in dataloader:
            images = images.to(device)
            features = model(images)
            all_features.append(features.cpu().numpy())
            all_targets.append(targets.numpy())

    features = np.vstack(all_features)
    targets = np.hstack(all_targets)
    return features, targets


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import pandas as pd
import numpy as np
import h5py
from PIL import Image
import io

# 数据集类
class CustomDataset(Dataset):
    def __init__(self, csv_file, img_hdf5_file, transform=None, is_test=False):
        self.data = pd.read_csv(csv_file)
        self.img_hdf5_file = img_hdf5_file
        self.transform = transform
        self.is_test = is_test
        
        # 处理图像ID
        self.image_ids = self.data['isic_id'].values
        
        # 处理结构化数据
        if 'target' in self.data.columns:
            self.structured_data = self.data.drop(columns=['isic_id', 'target'])
        else:
            self.structured_data = self.data.drop(columns=['isic_id'])
        
        for col in self.structured_data.columns:
            if self.structured_data[col].dtype == 'object':
                self.structured_data[col] = self.structured_data[col].astype(str).str.extract('(\d+)').astype(float)
        
        self.structured_data = self.structured_data.fillna(0).values.astype(np.float32)
        
        if not is_test:
            # 如果不是测试集，处理目标列
            self.targets = self.data['target'].values.astype(np.float32)
        else:
            self.targets = None

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        with h5py.File(self.img_hdf5_file, 'r') as hdf5_file:
            image_key = self.image_ids[idx]
            image_data = hdf5_file[image_key][()]
            
        image = Image.open(io.BytesIO(image_data))
        if self.transform:
            image = self.transform(image)
        
        structured_data = torch.tensor(self.structured_data[idx], dtype=torch.float32)
        if self.targets is not None:
            target = torch.tensor(self.targets[idx], dtype=torch.float32)
            return image, structured_data, target
        else:
            return image, structured_data, self.image_ids[idx]


# 定义LeNet模型提取图像特征
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten()
        )

    def forward(self, x):
        return self.conv_layers(x)

# 定义结合LeNet特征和结构化特征的XGBoost模型
class CombinedModel(nn.Module):
    def __init__(self, lenet_model, structured_data_dim):
        super(CombinedModel, self).__init__()
        self.lenet = lenet_model
        self.structured_data_dim = structured_data_dim

    def forward(self, image, structured_data):
        image_features = self.lenet(image).detach().cpu().numpy()
        combined_features = np.hstack((image_features, structured_data.detach().cpu().numpy()))
        return combined_features


# 数据变换
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # LeNet接受32x32图像输入
    transforms.ToTensor(),
])

# 加载数据集
train_dataset = CustomDataset(
    csv_file='data/train_metadata_aligned.csv',
    img_hdf5_file='data/train-image.hdf5',
    transform=transform
)-

# 划分训练集和验证集
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_set, val_set = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)

# 初始化模型
lenet_model = LeNet()
combined_model = CombinedModel(lenet_model, structured_data_dim=train_set[0][1].shape[0])

# XGBoost分类器
xgb_model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100)

# 训练与验证
best_auc = 0
for epoch in range(50):
    train_features = []
    train_labels = []

    # 训练过程
    for images, structured_data, targets in train_loader:
        features = combined_model(images, structured_data)
        train_features.append(features)
        train_labels.append(targets.numpy())
    
    train_features = np.vstack(train_features)
    train_labels = np.hstack(train_labels)

    xgb_model.fit(train_features, train_labels)

    # 验证过程
    val_features = []
    val_labels = []

    for images, structured_data, targets in val_loader:
        features = combined_model(images, structured_data)
        val_features.append(features)
        val_labels.append(targets.numpy())

    val_features = np.vstack(val_features)
    val_labels = np.hstack(val_labels)
    
    val_preds = xgb_model.predict_proba(val_features)[:, 1]
    auc = roc_auc_score(val_labels, val_preds)
    print(f"Epoch {epoch+1}/{50}, AUC: {auc}")

    # 保存最佳模型
    if auc > best_auc:
        best_auc = auc
        xgb_model.save_model(f"best_model_epoch_{epoch+1}.json")

# 测试过程
test_dataset = CustomDataset(
    csv_file='data/test_metadata_aligned.csv',
    img_hdf5_file='data/test-image.hdf5',
    transform=transform,
    is_test=True
)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

test_results = []
for images, structured_data, image_ids in test_loader:
    features = combined_model(images, structured_data)
    test_preds = xgb_model.predict_proba(features)[:, 1]

    for i in range(len(image_ids)):
        test_results.append({'isic_id': image_ids[i], 'target': test_preds[i]})

# 保存测试结果
test_df = pd.DataFrame(test_results)
test_df.to_csv('test_predictions.csv', index=False)


Epoch 1/50, AUC: 0.9166780029380396
Epoch 2/50, AUC: 0.9570450566478176
Epoch 3/50, AUC: 0.9209506369106312
Epoch 4/50, AUC: 0.9432427102408791
Epoch 5/50, AUC: 0.9348383072062464
Epoch 6/50, AUC: 0.9398981747932305
Epoch 7/50, AUC: 0.9397589197673717
Epoch 8/50, AUC: 0.9446336506147748
Epoch 9/50, AUC: 0.943432676634536
Epoch 10/50, AUC: 0.9305632584066166
Epoch 11/50, AUC: 0.9453428048216046


KeyboardInterrupt: 

In [18]:
test_dataset = CustomDataset(
    csv_file='data/test-metadata.csv',
    img_hdf5_file='data/test-image.hdf5',
    transform=transform,
    is_test=True
)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

test_results = []
for images, structured_data, image_ids in test_loader:
    features = combined_model(images, structured_data)
    test_preds = xgb_model.predict_proba(features)[:, 1]

    for i in range(len(image_ids)):
        test_results.append({'isic_id': image_ids[i], 'target': test_preds[i]})

# 保存测试结果
test_df = pd.DataFrame(test_results)
test_df.to_csv('test_predictions.csv', index=False)


In [ ]:
import torch
from torch.utils.data import DataLoader
import xgboost as xgb
import pandas as pd

# 假设你已经有了一个保存好的模型文件路径
model_path = 'best_model_epoch_1.json'  # 将 'X' 替换为保存模型的具体轮数

# 加载已经保存的XGBoost模型
xgb_model = xgb.Booster()
xgb_model.load_model(model_path)

# 创建测试集
test_dataset = CustomDataset(
    csv_file='data/test-metadata.csv',
    img_hdf5_file='data/test-image.hdf5',
    transform=transform,
    is_test=True
)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 进行测试并生成预测结果
test_results = []
for images, structured_data, image_ids in test_loader:
    features = combined_model(images, structured_data)  # 提取图像和结构化特征
    dtest = xgb.DMatrix(features)  # 创建DMatrix对象供XGBoost模型使用
    test_preds = xgb_model.predict(dtest)  # 使用加载的模型进行预测

    for i in range(len(image_ids)):
        test_results.append({'isic_id': image_ids[i], 'target': test_preds[i]})

# 保存测试结果到CSV文件
test_df = pd.DataFrame(test_results)
test_df.to_csv('test_predictions1.csv', index=False)


In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import pandas as pd
import numpy as np
import h5py
from PIL import Image
import io

# 数据集类
class CustomDataset(Dataset):
    def __init__(self, csv_file, img_hdf5_file, transform=None, is_test=False):
        self.data = pd.read_csv(csv_file)
        self.img_hdf5_file = img_hdf5_file
        self.transform = transform
        self.is_test = is_test
        
        # 处理图像ID
        self.image_ids = self.data['isic_id'].values
        
        # 处理结构化数据
        if 'target' in self.data.columns:
            self.structured_data = self.data.drop(columns=['isic_id', 'target'])
        else:
            self.structured_data = self.data.drop(columns=['isic_id'])
        
        for col in self.structured_data.columns:
            if self.structured_data[col].dtype == 'object':
                self.structured_data[col] = self.structured_data[col].astype(str).str.extract('(\d+)').astype(float)
        
        self.structured_data = self.structured_data.fillna(0).values.astype(np.float32)
        
        if not is_test:
            # 如果不是测试集，处理目标列
            self.targets = self.data['target'].values.astype(np.float32)
        else:
            self.targets = None

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        with h5py.File(self.img_hdf5_file, 'r') as hdf5_file:
            image_key = self.image_ids[idx]
            image_data = hdf5_file[image_key][()]
            
        image = Image.open(io.BytesIO(image_data))
        if self.transform:
            image = self.transform(image)
        
        structured_data = torch.tensor(self.structured_data[idx], dtype=torch.float32)
        if self.targets is not None:
            target = torch.tensor(self.targets[idx], dtype=torch.float32)
            return image, structured_data, target
        else:
            return image, structured_data, self.image_ids[idx]


# 定义LeNet模型提取图像特征
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten()
        )

    def forward(self, x):
        return self.conv_layers(x)

# 定义结合LeNet特征和结构化特征的XGBoost模型
class CombinedModel(nn.Module):
    def __init__(self, lenet_model, structured_data_dim):
        super(CombinedModel, self).__init__()
        self.lenet = lenet_model
        self.structured_data_dim = structured_data_dim

    def forward(self, image, structured_data):
        image_features = self.lenet(image).detach().cpu().numpy()
        combined_features = np.hstack((image_features, structured_data.detach().cpu().numpy()))
        return combined_features


# 数据变换
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # LeNet接受32x32图像输入
    transforms.ToTensor(),
])

# 加载数据集
train_dataset = CustomDataset(
    csv_file='data/train-metadata.csv',
    img_hdf5_file='data/train-image.hdf5',
    transform=transform
)

# 划分训练集和验证集
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_set, val_set = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)

# 初始化模型
lenet_model = LeNet()
combined_model = CombinedModel(lenet_model, structured_data_dim=train_set[0][1].shape[0])

# XGBoost分类器
xgb_model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100)

# 训练与验证
best_auc = 0
for epoch in range(50):
    train_features = []
    train_labels = []

    # 训练过程
    for images, structured_data, targets in train_loader:
        features = combined_model(images, structured_data)
        train_features.append(features)
        train_labels.append(targets.numpy())
    
    train_features = np.vstack(train_features)
    train_labels = np.hstack(train_labels)

    xgb_model.fit(train_features, train_labels)

    # 验证过程
    val_features = []
    val_labels = []

    for images, structured_data, targets in val_loader:
        features = combined_model(images, structured_data)
        val_features.append(features)
        val_labels.append(targets.numpy())

    val_features = np.vstack(val_features)
    val_labels = np.hstack(val_labels)
    
    val_preds = xgb_model.predict_proba(val_features)[:, 1]
    auc = roc_auc_score(val_labels, val_preds)
    print(f"Epoch {epoch+1}/{50}, AUC: {auc}")

    # 保存最佳模型
    if auc > best_auc:
        best_auc = auc
        xgb_model.save_model(f"0830best_model_epoch_{epoch+1}.json")

# 测试过程
test_dataset = CustomDataset(
    csv_file='data/test_metadata_aligned.csv',
    img_hdf5_file='data/test-image.hdf5',
    transform=transform,
    is_test=True
)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

test_results = []
for images, structured_data, image_ids in test_loader:
    features = combined_model(images, structured_data)
    test_preds = xgb_model.predict_proba(features)[:, 1]

    for i in range(len(image_ids)):
        test_results.append({'isic_id': image_ids[i], 'target': test_preds[i]})

# 保存测试结果
test_df = pd.DataFrame(test_results)
test_df.to_csv('test_predictions.csv', index=False)


/tmp/ipykernel_3751/3106650346.py:17: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = pd.read_csv(csv_file)


Epoch 1/50, AUC: 0.994080899132816
Epoch 2/50, AUC: 0.9935276177164156
Epoch 3/50, AUC: 0.9937861326639281
Epoch 4/50, AUC: 0.9937504754297886
Epoch 5/50, AUC: 0.9939923503347026
Epoch 6/50, AUC: 0.9943013796972463
Epoch 7/50, AUC: 0.99405296763274
Epoch 8/50, AUC: 0.9936440980146052
Epoch 9/50, AUC: 0.9934349089076526
Epoch 10/50, AUC: 0.9940202818347786
Epoch 11/50, AUC: 0.9938431842385517
Epoch 12/50, AUC: 0.9940927848775293
Epoch 13/50, AUC: 0.9946597349003499
Epoch 14/50, AUC: 0.9941575621862163
Epoch 15/50, AUC: 0.9933047600030427
Epoch 16/50, AUC: 0.9950448330290582
Epoch 17/50, AUC: 0.994568214666058
Epoch 18/50, AUC: 0.9944844201658299
Epoch 19/50, AUC: 0.9935638692377908
Epoch 20/50, AUC: 0.9944891744637152
Epoch 21/50, AUC: 0.9943245568994371
Epoch 22/50, AUC: 0.9934889890460976
Epoch 23/50, AUC: 0.9944499515061616
Epoch 24/50, AUC: 0.9946086261980831
Epoch 25/50, AUC: 0.9948172210178001
Epoch 26/50, AUC: 0.9942740424844059
Epoch 27/50, AUC: 0.9936084407804655
Epoch 28/50, A

FileNotFoundError: [Errno 2] No such file or directory: 'data/test_metadata_aligned.csv'

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import pandas as pd
import numpy as np
import h5py
from PIL import Image
import io
import torchvision.models as models

# 数据集类
class CustomDataset(Dataset):
    def __init__(self, csv_file, img_hdf5_file, transform=None, is_test=False):
        self.data = pd.read_csv(csv_file)
        self.img_hdf5_file = img_hdf5_file
        self.transform = transform
        self.is_test = is_test
        
        # 处理图像ID
        self.image_ids = self.data['isic_id'].values
        
        # 处理结构化数据
        if 'target' in self.data.columns:
            self.structured_data = self.data.drop(columns=['isic_id', 'target'])
        else:
            self.structured_data = self.data.drop(columns=['isic_id'])
        
        for col in self.structured_data.columns:
            if self.structured_data[col].dtype == 'object':
                self.structured_data[col] = self.structured_data[col].astype(str).str.extract('(\d+)').astype(float)
        
        self.structured_data = self.structured_data.fillna(0).values.astype(np.float32)
        
        if not is_test:
            # 如果不是测试集，处理目标列
            self.targets = self.data['target'].values.astype(np.float32)
        else:
            self.targets = None

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        with h5py.File(self.img_hdf5_file, 'r') as hdf5_file:
            image_key = self.image_ids[idx]
            image_data = hdf5_file[image_key][()]
            
        image = Image.open(io.BytesIO(image_data))
        if self.transform:
            image = self.transform(image)
        
        structured_data = torch.tensor(self.structured_data[idx], dtype=torch.float32)
        if self.targets is not None:
            target = torch.tensor(self.targets[idx], dtype=torch.float32)
            return image, structured_data, target
        else:
            return image, structured_data, self.image_ids[idx]

# 定义GoogLeNet模型以提取图像特征
class GoogleNetFeatureExtractor(nn.Module):
    def __init__(self):
        super(GoogleNetFeatureExtractor, self).__init__()
        googlenet = models.googlenet(pretrained=True)
        # 去掉最后的全连接层
        self.features = nn.Sequential(*list(googlenet.children())[:-2])
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.features(x)
        x = self.flatten(x)
        return x

# 定义结合GoogLeNet特征和结构化特征的XGBoost模型
class CombinedModel(nn.Module):
    def __init__(self, googlenet_model, structured_data_dim):
        super(CombinedModel, self).__init__()
        self.googlenet = googlenet_model
        self.structured_data_dim = structured_data_dim

    def forward(self, image, structured_data):
        image_features = self.googlenet(image).detach().cpu().numpy()
        combined_features = np.hstack((image_features, structured_data.detach().cpu().numpy()))
        return combined_features

# 数据变换
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # GoogLeNet接受224x224图像输入
    transforms.ToTensor(),
])

# 加载数据集
train_dataset = CustomDataset(
    csv_file='data/train-metadata.csv',
    img_hdf5_file='data/train-image.hdf5',
    transform=transform
)

# 划分训练集和验证集
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_set, val_set = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)

# 初始化模型
googlenet_model = GoogleNetFeatureExtractor()
combined_model = CombinedModel(googlenet_model, structured_data_dim=train_set[0][1].shape[0])

# XGBoost分类器
xgb_model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100)

# 训练与验证
best_auc = 0
for epoch in range(50):
    train_features = []
    train_labels = []

    # 训练过程
    for images, structured_data, targets in train_loader:
        features = combined_model(images, structured_data)
        train_features.append(features)
        train_labels.append(targets.numpy())
    
    train_features = np.vstack(train_features)
    train_labels = np.hstack(train_labels)

    xgb_model.fit(train_features, train_labels)

    # 验证过程
    val_features = []
    val_labels = []

    for images, structured_data, targets in val_loader:
        features = combined_model(images, structured_data)
        val_features.append(features)
        val_labels.append(targets.numpy())

    val_features = np.vstack(val_features)
    val_labels = np.hstack(val_labels)
    
    val_preds = xgb_model.predict_proba(val_features)[:, 1]
    auc = roc_auc_score(val_labels, val_preds)
    print(f"Epoch {epoch+1}/{50}, AUC: {auc}")

    # 保存最佳模型
    if auc > best_auc:
        best_auc = auc
        xgb_model.save_model(f"0904best_model_epoch_{epoch+1}.json")

# 测试过程
test_dataset = CustomDataset(
    csv_file='data/test_metadata_aligned.csv',
    img_hdf5_file='data/test-image.hdf5',
    transform=transform,
    is_test=True
)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

test_results = []
for images, structured_data, image_ids in test_loader:
    features = combined_model(images, structured_data)
    test_preds = xgb_model.predict_proba(features)[:, 1]

    for i in range(len(image_ids)):
        test_results.append({'isic_id': image_ids[i], 'target': test_preds[i]})

# 保存测试结果
test_df = pd.DataFrame(test_results)
test_df.to_csv('test_predictions.csv', index=False)


/tmp/ipykernel_520/1029356666.py:18: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = pd.read_csv(csv_file)
/usr/local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:09<00:00, 5.72MB/s]


Epoch 1/50, AUC: 0.9935246462802374


KeyboardInterrupt: 

In [1]:
#resnet18
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
from torchvision import models
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import pandas as pd
import numpy as np
import h5py
from PIL import Image
import io

# 数据集类
class CustomDataset(Dataset):
    def __init__(self, csv_file, img_hdf5_file, transform=None, is_test=False):
        self.data = pd.read_csv(csv_file)
        self.img_hdf5_file = img_hdf5_file
        self.transform = transform
        self.is_test = is_test
        
        # 处理图像ID
        self.image_ids = self.data['isic_id'].values
        
        # 处理结构化数据
        if 'target' in self.data.columns:
            self.structured_data = self.data.drop(columns=['isic_id', 'target'])
        else:
            self.structured_data = self.data.drop(columns=['isic_id'])
        
        for col in self.structured_data.columns:
            if self.structured_data[col].dtype == 'object':
                self.structured_data[col] = self.structured_data[col].astype(str).str.extract('(\d+)').astype(float)
        
        self.structured_data = self.structured_data.fillna(0).values.astype(np.float32)
        
        if not is_test:
            # 如果不是测试集，处理目标列
            self.targets = self.data['target'].values.astype(np.float32)
        else:
            self.targets = None

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        with h5py.File(self.img_hdf5_file, 'r') as hdf5_file:
            image_key = self.image_ids[idx]
            image_data = hdf5_file[image_key][()]
            
        image = Image.open(io.BytesIO(image_data))
        if self.transform:
            image = self.transform(image)
        
        structured_data = torch.tensor(self.structured_data[idx], dtype=torch.float32)
        if self.targets is not None:
            target = torch.tensor(self.targets[idx], dtype=torch.float32)
            return image, structured_data, target
        else:
            return image, structured_data, self.image_ids[idx]

# 定义ResNet18模型提取图像特征
class ResNet18FeatureExtractor(nn.Module):
    def __init__(self):
        super(ResNet18FeatureExtractor, self).__init__()
        resnet = models.resnet18(pretrained=True)
        self.features = nn.Sequential(*list(resnet.children())[:-1])  # 去掉最后的全连接层

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # 展平成1D向量
        return x

# 定义结合ResNet18特征和结构化特征的XGBoost模型
class CombinedModel(nn.Module):
    def __init__(self, resnet_model, structured_data_dim):
        super(CombinedModel, self).__init__()
        self.resnet = resnet_model
        self.structured_data_dim = structured_data_dim

    def forward(self, image, structured_data):
        image_features = self.resnet(image).detach().cpu().numpy()
        combined_features = np.hstack((image_features, structured_data.detach().cpu().numpy()))
        return combined_features

# 判断是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据变换
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet18接受224x224图像输入
    transforms.ToTensor(),
])

# 加载数据集
train_dataset = CustomDataset(
    csv_file='data/train-metadata.csv',
    img_hdf5_file='data/train-image.hdf5',
    transform=transform
)

# 划分训练集和验证集
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_set, val_set = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)

# 初始化模型
resnet_model = ResNet18FeatureExtractor().to(device)
combined_model = CombinedModel(resnet_model, structured_data_dim=train_set[0][1].shape[0])

# XGBoost分类器
xgb_model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100)

# 训练与验证
best_auc = 0
for epoch in range(50):
    train_features = []
    train_labels = []

    # 训练过程
    for images, structured_data, targets in train_loader:
        images, structured_data = images.to(device), structured_data.to(device)
        features = combined_model(images, structured_data)
        train_features.append(features)
        train_labels.append(targets.numpy())
    
    train_features = np.vstack(train_features)
    train_labels = np.hstack(train_labels)

    xgb_model.fit(train_features, train_labels)

    # 验证过程
    val_features = []
    val_labels = []

    for images, structured_data, targets in val_loader:
        images, structured_data = images.to(device), structured_data.to(device)
        features = combined_model(images, structured_data)
        val_features.append(features)
        val_labels.append(targets.numpy())

    val_features = np.vstack(val_features)
    val_labels = np.hstack(val_labels)
    
    val_preds = xgb_model.predict_proba(val_features)[:, 1]
    auc = roc_auc_score(val_labels, val_preds)
    print(f"Epoch {epoch+1}/{50}, AUC: {auc}")

    # 保存最佳模型
    if auc > best_auc:
        best_auc = auc
        xgb_model.save_model(f"resnet18_best_model_epoch_{epoch+1}.json")

# 测试过程
test_dataset = CustomDataset(
    csv_file='data/test_metadata_aligned.csv',
    img_hdf5_file='data/test-image.hdf5',
    transform=transform,
    is_test=True
)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

test_results = []
for images, structured_data, image_ids in test_loader:
    images, structured_data = images.to(device), structured_data.to(device)
    features = combined_model(images, structured_data)
    test_preds = xgb_model.predict_proba(features)[:, 1]

    for i in range(len(image_ids)):
        test_results.append({'isic_id': image_ids[i], 'target': test_preds[i]})

# 保存测试结果
test_df = pd.DataFrame(test_results)
test_df.to_csv('test_predictions.csv', index=False)


/tmp/ipykernel_413/2560157438.py:19: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = pd.read_csv(csv_file)
/usr/local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:05<00:00, 7.83MB/s]
/usr/local/lib

Epoch 1/50, AUC: 0.99106912984768
Epoch 2/50, AUC: 0.9934282771919316
Epoch 3/50, AUC: 0.9929475155393725
Epoch 4/50, AUC: 0.9933960950594516
Epoch 5/50, AUC: 0.994167152682545
Epoch 6/50, AUC: 0.994653168560815


KeyboardInterrupt: 

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
from torchvision import models
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import pandas as pd
import numpy as np
import h5py
from PIL import Image
import io
from sklearn.preprocessing import StandardScaler

# 数据集类
class CustomDataset(Dataset):
    def __init__(self, csv_file, img_hdf5_file, transform=None, is_test=False):
        self.data = pd.read_csv(csv_file)
        self.img_hdf5_file = img_hdf5_file
        self.transform = transform
        self.is_test = is_test
        
        # 处理图像ID
        self.image_ids = self.data['isic_id'].values
        
        # 处理结构化数据
        if 'target' in self.data.columns:
            self.structured_data = self.data.drop(columns=['isic_id', 'target'])
        else:
            self.structured_data = self.data.drop(columns=['isic_id'])
        
        # 特征工程：将字符串类型的列转换为数字
        for col in self.structured_data.columns:
            if self.structured_data[col].dtype == 'object':
                self.structured_data[col] = self.structured_data[col].astype(str).str.extract('(\d+)').astype(float)
        
        # 处理缺失值
        self.structured_data = self.structured_data.fillna(0).values.astype(np.float32)
        
        # 特征标准化
        self.scaler = StandardScaler()
        self.structured_data = self.scaler.fit_transform(self.structured_data)
        
        if not is_test:
            # 如果不是测试集，处理目标列
            self.targets = self.data['target'].values.astype(np.float32)
        else:
            self.targets = None

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        with h5py.File(self.img_hdf5_file, 'r') as hdf5_file:
            image_key = self.image_ids[idx]
            image_data = hdf5_file[image_key][()]
            
        image = Image.open(io.BytesIO(image_data))
        if self.transform:
            image = self.transform(image)
        
        structured_data = torch.tensor(self.structured_data[idx], dtype=torch.float32)
        if self.targets is not None:
            target = torch.tensor(self.targets[idx], dtype=torch.float32)
            return image, structured_data, target
        else:
            return image, structured_data, self.image_ids[idx]

# 定义MobileNetV1模型提取图像特征
class MobileNetV1FeatureExtractor(nn.Module):
    def __init__(self):
        super(MobileNetV1FeatureExtractor, self).__init__()
        mobilenet = models.mobilenet_v2(pretrained=True)
        self.features = mobilenet.features
        self.pool = nn.AdaptiveAvgPool2d(1)  # 自适应平均池化
        
    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        return x

# 定义结合MobileNetV1特征和结构化特征的XGBoost模型
class CombinedModel(nn.Module):
    def __init__(self, mobilenet_model, structured_data_dim):
        super(CombinedModel, self).__init__()
        self.mobilenet = mobilenet_model
        self.structured_data_dim = structured_data_dim

    def forward(self, image, structured_data):
        image_features = self.mobilenet(image).detach().cpu().numpy()
        combined_features = np.hstack((image_features, structured_data.detach().cpu().numpy()))
        return combined_features

# 判断是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据变换
transform = transforms.Compose([
    transforms.Resize((15, 15)),  # MobileNetV1接受15x15图像输入
    transforms.ToTensor(),
])

# 加载数据集
train_dataset = CustomDataset(
    csv_file='data/train-metadata.csv',
    img_hdf5_file='data/train-image.hdf5',
    transform=transform
)

# 划分训练集和验证集
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_set, val_set = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)

# 初始化模型
mobilenet_model = MobileNetV1FeatureExtractor().to(device)
combined_model = CombinedModel(mobilenet_model, structured_data_dim=train_set[0][1].shape[0])

# XGBoost分类器
xgb_model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100)

# 训练与验证
best_auc = 0
for epoch in range(50):
    train_features = []
    train_labels = []

    # 训练过程
    for images, structured_data, targets in train_loader:
        images, structured_data = images.to(device), structured_data.to(device)
        features = combined_model(images, structured_data)
        train_features.append(features)
        train_labels.append(targets.numpy())
    
    train_features = np.vstack(train_features)
    train_labels = np.hstack(train_labels)

    xgb_model.fit(train_features, train_labels)

    # 验证过程
    val_features = []
    val_labels = []

    for images, structured_data, targets in val_loader:
        images, structured_data = images.to(device), structured_data.to(device)
        features = combined_model(images, structured_data)
        val_features.append(features)
        val_labels.append(targets.numpy())

    val_features = np.vstack(val_features)
    val_labels = np.hstack(val_labels)
    
    val_preds = xgb_model.predict_proba(val_features)[:, 1]
    auc = roc_auc_score(val_labels, val_preds)
    print(f"Epoch {epoch+1}/{50}, AUC: {auc}")

    # 保存最佳模型
    if auc > best_auc:
        best_auc = auc
        xgb_model.save_model(f"mobilenet_best_model_epoch_{epoch+1}.json")

# 测试过程
test_dataset = CustomDataset(
    csv_file='data/test_metadata_aligned.csv',
    img_hdf5_file='data/test-image.hdf5',
    transform=transform,
    is_test=True
)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

test_results = []
for images, structured_data, image_ids in test_loader:
    images, structured_data = images.to(device), structured_data.to(device)
    features = combined_model(images, structured_data)
    test_preds = xgb_model.predict_proba(features)[:, 1]

    for i in range(len(image_ids)):
        test_results.append({'isic_id': image_ids[i], 'target': test_preds[i]})

# 保存测试结果
test_df = pd.DataFrame(test_results)
test_df.to_csv('test_predictions.csv', index=False)


/tmp/ipykernel_413/3569154635.py:18: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = pd.read_csv(csv_file)
/usr/local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:01<00:00, 8.85MB/s

Epoch 1/50, AUC: 0.9943342069137652
Epoch 2/50, AUC: 0.9945107493006791
Epoch 3/50, AUC: 0.9929340431554821
Epoch 4/50, AUC: 0.9944930950619877
Epoch 5/50, AUC: 0.9944291745425878
Epoch 6/50, AUC: 0.9952473571909062
Epoch 7/50, AUC: 0.9936956104866177
Epoch 8/50, AUC: 0.9949253195265012
Epoch 9/50, AUC: 0.9949557578690725
Epoch 10/50, AUC: 0.9961665951365617
Epoch 11/50, AUC: 0.9952875358031006
Epoch 12/50, AUC: 0.9951170810847008
Epoch 13/50, AUC: 0.9936956104866178
Epoch 14/50, AUC: 0.9957995087251509
Epoch 15/50, AUC: 0.9941381839876056
Epoch 16/50, AUC: 0.9939427698282973
Epoch 17/50, AUC: 0.994662941013536
Epoch 18/50, AUC: 0.9945728435195247
Epoch 19/50, AUC: 0.9945831925559989
Epoch 20/50, AUC: 0.996537334149081
Epoch 21/50, AUC: 0.995480514895003
Epoch 22/50, AUC: 0.9955736562232713
Epoch 23/50, AUC: 0.9948108713584329


KeyboardInterrupt: 

In [ ]:
from sklearn.svm import SVC

# 定义MobileNetV1模型提取图像特征
class MobileNetV1FeatureExtractor(nn.Module):
    def __init__(self):
        super(MobileNetV1FeatureExtractor, self).__init__()
        mobilenet = models.mobilenet_v2(pretrained=True)
        self.features = mobilenet.features
        self.pool = nn.AdaptiveAvgPool2d(1)  # 自适应平均池化

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        return x

# 定义结合MobileNetV1特征和结构化特征的SVM和XGBoost模型
class CombinedModel(nn.Module):
    def __init__(self, mobilenet_model, structured_data_dim):
        super(CombinedModel, self).__init__()
        self.mobilenet = mobilenet_model
        self.structured_data_dim = structured_data_dim
        self.svm = SVC(probability=True)  # 定义SVM分类器

    def forward(self, image, structured_data):
        # 提取图像特征
        with torch.no_grad():
            image_features = self.mobilenet(image).detach().cpu().numpy()
        return image_features, structured_data.detach().cpu().numpy()

# 初始化模型
mobilenet_model = MobileNetV1FeatureExtractor().to(device)
combined_model = CombinedModel(mobilenet_model, structured_data_dim=train_set[0][1].shape[0])

# XGBoost分类器
xgb_model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100)

# SVM模型初始化
svm_model = SVC(probability=True)

# 训练与验证
best_auc = 0
for epoch in range(50):
    train_image_features = []
    train_structured_features = []
    train_labels = []

    # 训练过程
    for images, structured_data, targets in train_loader:
        images, structured_data = images.to(device), structured_data.to(device)
        image_features, structured_features = combined_model(images, structured_data)
        
        train_image_features.append(image_features)
        train_structured_features.append(structured_features)
        train_labels.append(targets.numpy())

    train_image_features = np.vstack(train_image_features)
    train_structured_features = np.vstack(train_structured_features)
    train_labels = np.hstack(train_labels)

    # 先用SVM拟合图像特征
    svm_model.fit(train_image_features, train_labels)

    # 获取SVM预测概率作为新的图像特征
    svm_train_probs = svm_model.predict_proba(train_image_features)[:, 1]

    # 将SVM预测的概率与结构化特征组合
    combined_train_features = np.hstack((svm_train_probs.reshape(-1, 1), train_structured_features))

    # 用XGBoost拟合组合特征
    xgb_model.fit(combined_train_features, train_labels)

    # 验证过程
    val_image_features = []
    val_structured_features = []
    val_labels = []

    for images, structured_data, targets in val_loader:
        images, structured_data = images.to(device), structured_data.to(device)
        image_features, structured_features = combined_model(images, structured_data)

        val_image_features.append(image_features)
        val_structured_features.append(structured_features)
        val_labels.append(targets.numpy())

    val_image_features = np.vstack(val_image_features)
    val_structured_features = np.vstack(val_structured_features)
    val_labels = np.hstack(val_labels)

    # 使用SVM预测验证集图像特征
    svm_val_probs = svm_model.predict_proba(val_image_features)[:, 1]

    # 将SVM预测的概率与结构化特征组合
    combined_val_features = np.hstack((svm_val_probs.reshape(-1, 1), val_structured_features))

    # XGBoost在验证集上预测
    val_preds = xgb_model.predict_proba(combined_val_features)[:, 1]
    auc = roc_auc_score(val_labels, val_preds)
    print(f"Epoch {epoch+1}/{50}, AUC: {auc}")

    # 保存最佳模型
    if auc > best_auc:
        best_auc = auc
        xgb_model.save_model(f"svm_xgb_best_model_epoch_{epoch+1}.json")

# 测试过程
test_dataset = CustomDataset(
    csv_file='data/test_metadata_aligned.csv',
    img_hdf5_file='data/test-image.hdf5',
    transform=transform,
    is_test=True
)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

test_results = []
for images, structured_data, image_ids in test_loader:
    images, structured_data = images.to(device), structured_data.to(device)
    image_features, structured_features = combined_model(images, structured_data)
    
    # SVM预测测试集图像特征
    svm_test_probs = svm_model.predict_proba(image_features)[:, 1]

    # 将SVM预测的概率与结构化特征组合
    combined_test_features = np.hstack((svm_test_probs.reshape(-1, 1), structured_features))

    # XGBoost在测试集上预测
    test_preds = xgb_model.predict_proba(combined_test_features)[:, 1]

    for i in range(len(image_ids)):
        test_results.append({'isic_id': image_ids[i], 'target': test_preds[i]})

# 保存测试结果
test_df = pd.DataFrame(test_results)
test_df.to_csv('test_predictions.csv', index=False)


/usr/local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
from torchvision import models
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import pandas as pd
import numpy as np
import h5py
from PIL import Image
import io
from sklearn.preprocessing import StandardScaler

# 数据集类
class CustomDataset(Dataset):
    def __init__(self, csv_file, img_hdf5_file, transform=None, is_test=False):
        self.data = pd.read_csv(csv_file)
        self.img_hdf5_file = img_hdf5_file
        self.transform = transform
        self.is_test = is_test
        
        # 处理图像ID
        self.image_ids = self.data['isic_id'].values
        
        # 处理结构化数据
        if 'target' in self.data.columns:
            self.structured_data = self.data.drop(columns=['isic_id', 'target'])
        else:
            self.structured_data = self.data.drop(columns=['isic_id'])
        
        # 特征工程：将字符串类型的列转换为数字
        for col in self.structured_data.columns:
            if self.structured_data[col].dtype == 'object':
                self.structured_data[col] = self.structured_data[col].astype(str).str.extract('(\d+)').astype(float)
        
        # 处理缺失值
        self.structured_data = self.structured_data.fillna(0).values.astype(np.float32)
        
        # 特征标准化
        self.scaler = StandardScaler()
        self.structured_data = self.scaler.fit_transform(self.structured_data)
        
        if not is_test:
            # 如果不是测试集，处理目标列
            self.targets = self.data['target'].values.astype(np.float32)
        else:
            self.targets = None

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        with h5py.File(self.img_hdf5_file, 'r') as hdf5_file:
            image_key = self.image_ids[idx]
            image_data = hdf5_file[image_key][()]
            
        image = Image.open(io.BytesIO(image_data))
        if self.transform:
            image = self.transform(image)
        
        structured_data = torch.tensor(self.structured_data[idx], dtype=torch.float32)
        if self.targets is not None:
            target = torch.tensor(self.targets[idx], dtype=torch.float32)
            return image, structured_data, target
        else:
            return image, structured_data, self.image_ids[idx]

# 定义MobileNetV1模型提取图像特征
class MobileNetV1FeatureExtractor(nn.Module):
    def __init__(self, weights_path=None):
        super(MobileNetV1FeatureExtractor, self).__init__()
        mobilenet = models.mobilenet_v2(pretrained=False)
        self.features = mobilenet.features
        self.pool = nn.AdaptiveAvgPool2d(1)  # 自适应平均池化
        
        # 加载本地权重
        if weights_path:
            self.load_state_dict(torch.load(weights_path))

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        return x

# 定义结合MobileNetV1特征和结构化特征的XGBoost模型
class CombinedModel(nn.Module):
    def __init__(self, mobilenet_model, structured_data_dim):
        super(CombinedModel, self).__init__()
        self.mobilenet = mobilenet_model
        self.structured_data_dim = structured_data_dim

    def forward(self, image, structured_data):
        image_features = self.mobilenet(image).detach().cpu().numpy()
        combined_features = np.hstack((image_features, structured_data.detach().cpu().numpy()))
        return combined_features

# 判断是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据变换
transform = transforms.Compose([
    transforms.Resize((15, 15)),  # MobileNetV1接受15x15图像输入
    transforms.ToTensor(),
])

# 加载数据集
train_dataset = CustomDataset(
    csv_file='data/train-metadata.csv',
    img_hdf5_file='data/train-image.hdf5',
    transform=transform
)

# 划分训练集和验证集
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_set, val_set = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)

# 初始化模型
mobilenet_weights_path = 'data/mobilenet_v2-b0353104.pth'
mobilenet_model = MobileNetV1FeatureExtractor(weights_path=mobilenet_weights_path).to(device)
combined_model = CombinedModel(mobilenet_model, structured_data_dim=train_set[0][1].shape[0])

# XGBoost分类器
xgb_model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100)

# 训练与验证
best_auc = 0
for epoch in range(50):
    train_features = []
    train_labels = []

    # 训练过程
    for images, structured_data, targets in train_loader:
        images, structured_data = images.to(device), structured_data.to(device)
        features = combined_model(images, structured_data)
        train_features.append(features)
        train_labels.append(targets.numpy())
    
    train_features = np.vstack(train_features)
    train_labels = np.hstack(train_labels)

    xgb_model.fit(train_features, train_labels)

    # 验证过程
    val_features = []
    val_labels = []

    for images, structured_data, targets in val_loader:
        images, structured_data = images.to(device), structured_data.to(device)
        features = combined_model(images, structured_data)
        val_features.append(features)
        val_labels.append(targets.numpy())

    val_features = np.vstack(val_features)
    val_labels = np.hstack(val_labels)
    
    val_preds = xgb_model.predict_proba(val_features)[:, 1]
    auc = roc_auc_score(val_labels, val_preds)
    print(f"Epoch {epoch+1}/{50}, AUC: {auc}")

    # 保存最佳模型
    if auc > best_auc:
        best_auc = auc
        xgb_model.save_model(f"mobilenet_best_model_epoch_{epoch+1}.json")

# 测试过程
test_dataset = CustomDataset(
    csv_file='data/test-metadata-aligned.csv',
    img_hdf5_file='data/test-image.hdf5',
    transform=transform,
    is_test=True
)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

test_results = []
for images, structured_data, image_ids in test_loader:
    images, structured_data = images.to(device), structured_data.to(device)
    features = combined_model(images, structured_data)
    test_preds = xgb_model.predict_proba(features)[:, 1]

    for i in range(len(image_ids)):
        test_results.append({'isic_id': image_ids[i], 'target': test_preds[i]})

# 保存测试结果
test_df = pd.DataFrame(test_results)
test_df.to_csv('test_predictions.csv', index=False)


In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
from torchvision import models
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import pandas as pd
import numpy as np
import h5py
from PIL import Image
import io
from sklearn.preprocessing import StandardScaler

# 数据集类
class CustomDataset(Dataset):
    def __init__(self, csv_file, img_hdf5_file, transform=None, is_test=False):
        self.data = pd.read_csv(csv_file)
        self.img_hdf5_file = img_hdf5_file
        self.transform = transform
        self.is_test = is_test
        
        # 处理图像ID
        self.image_ids = self.data['isic_id'].values
        
        # 处理结构化数据
        if 'target' in self.data.columns:
            self.structured_data = self.data.drop(columns=['isic_id', 'target'])
        else:
            self.structured_data = self.data.drop(columns=['isic_id'])
        
        # 特征工程：将字符串类型的列转换为数字
        for col in self.structured_data.columns:
            if self.structured_data[col].dtype == 'object':
                self.structured_data[col] = self.structured_data[col].astype(str).str.extract('(\d+)').astype(float)
        
        # 处理缺失值
        self.structured_data = self.structured_data.fillna(0).values.astype(np.float32)
        
        # 特征标准化
        self.scaler = StandardScaler()
        self.structured_data = self.scaler.fit_transform(self.structured_data)
        
        if not is_test:
            # 如果不是测试集，处理目标列
            self.targets = self.data['target'].values.astype(np.float32)
        else:
            self.targets = None

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        with h5py.File(self.img_hdf5_file, 'r') as hdf5_file:
            image_key = self.image_ids[idx]
            image_data = hdf5_file[image_key][()]
            
        image = Image.open(io.BytesIO(image_data))
        if self.transform:
            image = self.transform(image)
        
        structured_data = torch.tensor(self.structured_data[idx], dtype=torch.float32)
        if self.targets is not None:
            target = torch.tensor(self.targets[idx], dtype=torch.float32)
            return image, structured_data, target
        else:
            return image, structured_data, self.image_ids[idx]

# 定义MobileNetV1模型提取图像特征
class MobileNetV1FeatureExtractor(nn.Module):
    def __init__(self, weights_path=None):
        super(MobileNetV1FeatureExtractor, self).__init__()
        mobilenet = models.mobilenet_v2(pretrained=False)
        self.features = mobilenet.features
        self.pool = nn.AdaptiveAvgPool2d(1)  # 自适应平均池化
        
        # 加载本地权重
        if weights_path:
            self.load_state_dict(torch.load(weights_path))

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        return x

# 定义结合MobileNetV1特征和结构化特征的XGBoost模型
class CombinedModel(nn.Module):
    def __init__(self, mobilenet_model, structured_data_dim):
        super(CombinedModel, self).__init__()
        self.mobilenet = mobilenet_model
        self.structured_data_dim = structured_data_dim

    def forward(self, image, structured_data):
        image_features = self.mobilenet(image).detach().cpu().numpy()
        combined_features = np.hstack((image_features, structured_data.detach().cpu().numpy()))
        return combined_features

# 判断是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据变换
transform = transforms.Compose([
    transforms.Resize((15, 15)),  # MobileNetV1接受15x15图像输入
    transforms.ToTensor(),
])

# 加载数据集
train_dataset = CustomDataset(
    csv_file='data/train-metadata.csv',
    img_hdf5_file='data/train-image.hdf5',
    transform=transform
)

# 划分训练集和验证集
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_set, val_set = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)

# 初始化模型
mobilenet_weights_path = 'data/mobilenet_v2-b0353104.pth'
mobilenet_model = MobileNetV1FeatureExtractor(weights_path=mobilenet_weights_path).to(device)
combined_model = CombinedModel(mobilenet_model, structured_data_dim=train_set[0][1].shape[0])

# XGBoost分类器
xgb_model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100)

# 训练与验证
best_auc = 0
for epoch in range(1):
    train_features = []
    train_labels = []

    # 训练过程
    for images, structured_data, targets in train_loader:
        images, structured_data = images.to(device), structured_data.to(device)
        features = combined_model(images, structured_data)
        train_features.append(features)
        train_labels.append(targets.numpy())
    
    train_features = np.vstack(train_features)
    train_labels = np.hstack(train_labels)

    xgb_model.fit(train_features, train_labels)

    # 验证过程
    val_features = []
    val_labels = []

    for images, structured_data, targets in val_loader:
        images, structured_data = images.to(device), structured_data.to(device)
        features = combined_model(images, structured_data)
        val_features.append(features)
        val_labels.append(targets.numpy())

    val_features = np.vstack(val_features)
    val_labels = np.hstack(val_labels)
    
    val_preds = xgb_model.predict_proba(val_features)[:, 1]
    auc = roc_auc_score(val_labels, val_preds)
    print(f"Epoch {epoch+1}/50, AUC: {auc}")

    # 保存最佳模型和MobileNetV1权重
    if auc > best_auc:
        best_auc = auc
        xgb_model.save_model(f"best_xgb_model.json")
        torch.save(mobilenet_model.state_dict(), f"best_mobilenet_v1_weights.pth")
        print(f'Best model saved at epoch {epoch + 1}')

# 测试过程
test_dataset = CustomDataset(
    csv_file='data/test-metadata-aligned.csv',
    img_hdf5_file='data/test-image.hdf5',
    transform=transform,
    is_test=True
)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 加载保存的最佳 XGBoost 模型
xgb_model = xgb.XGBClassifier()
xgb_model.load_model('best_xgb_model.json')

# 加载保存的 MobileNetV1 模型权重
mobilenet_model = MobileNetV1FeatureExtractor().to(device)
mobilenet_model.load_state_dict(torch.load('best_mobilenet_v1_weights.pth'))
mobilenet_model.eval()  # 设置模型为评估模式

# 测试阶段生成预测
test_results = []
with torch.no_grad():
    for images, structured_data, image_ids in test_loader:
        images, structured_data = images.to(device), structured_data.to(device)
        
        # 使用加载的MobileNetV1提取图像特征
        image_features = mobilenet_model(images).cpu().numpy()
        
        # 结合图像特征和结构化数据
        combined_features = np.hstack((image_features, structured_data.cpu().numpy()))
        
        # 使用 XGBoost 模型进行预测
        test_preds = xgb_model.predict_proba(combined_features)[:, 1]  # 预测概率

        for i in range(len(image_ids)):
            test_results.append({'isic_id': image_ids[i], 'target': test_preds[i]})

# 保存测试结果
test_df = pd.DataFrame(test_results)
test_df.to_csv('test_predictions.csv', index=False)

print("测试完成，预测结果已保存至 test_predictions.csv")


/tmp/ipykernel_282/1327625922.py:18: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = pd.read_csv(csv_file)
/usr/local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


RuntimeError: Error(s) in loading state_dict for MobileNetV1FeatureExtractor:
	Unexpected key(s) in state_dict: "classifier.1.weight", "classifier.1.bias". 

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import h5py
from PIL import Image
import io
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

# 数据集类
class CustomDataset(Dataset):
    def __init__(self, csv_file, img_hdf5_file, transform=None, is_test=False):
        self.data = pd.read_csv(csv_file)
        self.img_hdf5_file = img_hdf5_file
        self.transform = transform
        self.is_test = is_test
        
        # 处理图像ID
        self.image_ids = self.data['isic_id'].values
        
        # 处理结构化数据
        if 'target' in self.data.columns:
            self.structured_data = self.data.drop(columns=['isic_id', 'target'])
        else:
            self.structured_data = self.data.drop(columns=['isic_id'])
        
        # 特征工程：将字符串类型的列转换为数字
        for col in self.structured_data.columns:
            if self.structured_data[col].dtype == 'object':
                self.structured_data[col] = self.structured_data[col].astype(str).str.extract('(\d+)').astype(float)
        
        # 处理缺失值
        self.structured_data = self.structured_data.fillna(0).values.astype(np.float32)
        
        # 特征标准化
        self.scaler = StandardScaler()
        self.structured_data = self.scaler.fit_transform(self.structured_data)
        
        if not is_test:
            # 如果不是测试集，处理目标列
            self.targets = self.data['target'].values.astype(np.float32)
        else:
            self.targets = None

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        with h5py.File(self.img_hdf5_file, 'r') as hdf5_file:
            image_key = self.image_ids[idx]
            image_data = hdf5_file[image_key][()]
            
        image = Image.open(io.BytesIO(image_data))
        if self.transform:
            image = self.transform(image)
        
        structured_data = torch.tensor(self.structured_data[idx], dtype=torch.float32)
        if self.targets is not None:
            target = torch.tensor(self.targets[idx], dtype=torch.float32)
            return image, structured_data, target
        else:
            return image, structured_data, self.image_ids[idx]

# 深度可分离卷积（Depthwise Separable Convolution）
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, expand_ratio=1):
        super(DepthwiseSeparableConv, self).__init__()
        hidden_channels = in_channels * expand_ratio

        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, hidden_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False),
            nn.BatchNorm2d(hidden_channels),
            nn.ReLU6(inplace=True),
            nn.Conv2d(hidden_channels, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )

    def forward(self, x):
        return self.conv(x)

# MobileNetV2 的基本块（Block）
class MobileNetV2Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride, expand_ratio):
        super(MobileNetV2Block, self).__init__()
        hidden_channels = in_channels * expand_ratio

        self.block = nn.Sequential(
            nn.Conv2d(in_channels, hidden_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(hidden_channels),
            nn.ReLU6(inplace=True),
            DepthwiseSeparableConv(hidden_channels, out_channels, stride=stride, expand_ratio=expand_ratio)
        )

        self.use_residual = stride == 1 and in_channels == out_channels

    def forward(self, x):
        if self.use_residual:
            return x + self.block(x)
        else:
            return self.block(x)

# MobileNetV2 模型
class MobileNetV2(nn.Module):
    def __init__(self, num_classes=1000):
        super(MobileNetV2, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU6(inplace=True),
            
            MobileNetV2Block(32, 16, stride=1, expand_ratio=1),
            MobileNetV2Block(16, 24, stride=2, expand_ratio=6),
            MobileNetV2Block(24, 24, stride=1, expand_ratio=6),
            MobileNetV2Block(24, 32, stride=2, expand_ratio=6),
            MobileNetV2Block(32, 32, stride=1, expand_ratio=6),
            MobileNetV2Block(32, 64, stride=2, expand_ratio=6),
            MobileNetV2Block(64, 64, stride=1, expand_ratio=6),
            MobileNetV2Block(64, 96, stride=1, expand_ratio=6),
            MobileNetV2Block(96, 96, stride=1, expand_ratio=6),
            MobileNetV2Block(96, 160, stride=2, expand_ratio=6),
            MobileNetV2Block(160, 160, stride=1, expand_ratio=6),
            MobileNetV2Block(160, 320, stride=1, expand_ratio=6),

            nn.Conv2d(320, 1280, kernel_size=1, bias=False),
            nn.BatchNorm2d(1280),
            nn.ReLU6(inplace=True),

            nn.AdaptiveAvgPool2d(1)
        )

        self.fc = nn.Linear(1280, num_classes)

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# 定义结合MobileNetV2特征和结构化特征的XGBoost模型
class CombinedModel(nn.Module):
    def __init__(self, mobilenet_model, structured_data_dim):
        super(CombinedModel, self).__init__()
        self.mobilenet = mobilenet_model
        self.structured_data_dim = structured_data_dim

    def forward(self, image, structured_data):
        image_features = self.mobilenet(image).detach().cpu().numpy()
        combined_features = np.hstack((image_features, structured_data.detach().cpu().numpy()))
        return combined_features

# 判断是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据变换
transform = transforms.Compose([
    transforms.Resize((15, 15)),  # 修改为15x15图像尺寸
    transforms.ToTensor(),
])

# 加载数据集
train_dataset = CustomDataset(
    csv_file='data/train-metadata.csv',
    img_hdf5_file='data/train-image.hdf5',
    transform=transform
)

# 划分训练集和验证集
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_set, val_set = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)

# 初始化模型
mobilenet_model = MobileNetV2(num_classes=1280).to(device)
combined_model = CombinedModel(mobilenet_model, structured_data_dim=train_set[0][1].shape[0])

# XGBoost分类器
xgb_model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100)

# 训练与验证
best_auc = 0
for epoch in range(10):  # 这里只用1个epoch演示，实际应使用更多
    train_features = []
    train_labels = []

    # 训练过程
    for images, structured_data, targets in train_loader:
        images, structured_data = images.to(device), structured_data.to(device)
        features = combined_model(images, structured_data)
        train_features.append(features)
        train_labels.append(targets.numpy())
    
    train_features = np.vstack(train_features)
    train_labels = np.hstack(train_labels)

    xgb_model.fit(train_features, train_labels)

    # 验证过程
    val_features = []
    val_labels = []

    for images, structured_data, targets in val_loader:
        images, structured_data = images.to(device), structured_data.to(device)
        features = combined_model(images, structured_data)
        val_features.append(features)
        val_labels.append(targets.numpy())

    val_features = np.vstack(val_features)
    val_labels = np.hstack(val_labels)
    
    val_preds = xgb_model.predict_proba(val_features)[:, 1]
    auc = roc_auc_score(val_labels, val_preds)
    print(f"Epoch {epoch+1}: AUC = {auc:.4f}")

    # 保存最佳模型
    if auc > best_auc:
        best_auc = auc
        torch.save(combined_model.state_dict(), 'best_combined_model.pth')
        xgb_model.save_model('best_xgb_model.json')

# 测试过程
test_dataset = CustomDataset(
    csv_file='data/test-metadata.csv',
    img_hdf5_file='data/test-image.hdf5',
    transform=transform,
    is_test=True
)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 加载最佳模型
combined_model.load_state_dict(torch.load('best_combined_model.pth'))
xgb_model.load_model('best_xgb_model.json')

test_image_names = []
test_predictions = []

for images, structured_data, image_ids in test_loader:
    images, structured_data = images.to(device), structured_data.to(device)
    features = combined_model(images, structured_data)
    preds = xgb_model.predict_proba(features)[:, 1]
    test_image_names.extend(image_ids.numpy())
    test_predictions.extend(preds)

# 保存测试结果
results = pd.DataFrame({'image_id': test_image_names, 'prediction': test_predictions})
results.to_csv('test_predictions.csv', index=False)

print("Testing completed. Predictions saved to test_predictions.csv")


/tmp/ipykernel_282/2795419060.py:19: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = pd.read_csv(csv_file)


Epoch 1: AUC = 0.9948
Epoch 2: AUC = 0.9930
Epoch 3: AUC = 0.9947
Epoch 4: AUC = 0.9932
Epoch 5: AUC = 0.9938
Epoch 6: AUC = 0.9943
Epoch 7: AUC = 0.9946
Epoch 8: AUC = 0.9941
Epoch 9: AUC = 0.9941
Epoch 10: AUC = 0.9934


ValueError: Feature shape mismatch, expected: 1333, got 1323

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import h5py
from PIL import Image
import io
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

# 数据集类
class CustomDataset(Dataset):
    def __init__(self, csv_file, img_hdf5_file, transform=None, is_test=False):
        self.data = pd.read_csv(csv_file)
        self.img_hdf5_file = img_hdf5_file
        self.transform = transform
        self.is_test = is_test
        
        # 处理图像ID
        self.image_ids = self.data['isic_id'].values
        
        # 处理结构化数据
        if 'target' in self.data.columns:
            self.structured_data = self.data.drop(columns=['isic_id', 'target'])
        else:
            self.structured_data = self.data.drop(columns=['isic_id'])
        
        # 特征工程：将字符串类型的列转换为数字
        for col in self.structured_data.columns:
            if self.structured_data[col].dtype == 'object':
                self.structured_data[col] = self.structured_data[col].astype(str).str.extract('(\d+)').astype(float)
        
        # 处理缺失值
        self.structured_data = self.structured_data.fillna(0).values.astype(np.float32)
        
        # 特征标准化
        self.scaler = StandardScaler()
        self.structured_data = self.scaler.fit_transform(self.structured_data)
        
        if not is_test:
            # 如果不是测试集，处理目标列
            self.targets = self.data['target'].values.astype(np.float32)
        else:
            self.targets = None

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        with h5py.File(self.img_hdf5_file, 'r') as hdf5_file:
            image_key = self.image_ids[idx]
            image_data = hdf5_file[image_key][()]
            
        image = Image.open(io.BytesIO(image_data))
        if self.transform:
            image = self.transform(image)
        
        structured_data = torch.tensor(self.structured_data[idx], dtype=torch.float32)
        if self.targets is not None:
            target = torch.tensor(self.targets[idx], dtype=torch.float32)
            return image, structured_data, target
        else:
            return image, structured_data, self.image_ids[idx]

# 深度可分离卷积（Depthwise Separable Convolution）
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, expand_ratio=1):
        super(DepthwiseSeparableConv, self).__init__()
        hidden_channels = in_channels * expand_ratio

        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, hidden_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False),
            nn.BatchNorm2d(hidden_channels),
            nn.ReLU6(inplace=True),
            nn.Conv2d(hidden_channels, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )

    def forward(self, x):
        return self.conv(x)

# MobileNetV2 的基本块（Block）
class MobileNetV2Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride, expand_ratio):
        super(MobileNetV2Block, self).__init__()
        hidden_channels = in_channels * expand_ratio

        self.block = nn.Sequential(
            nn.Conv2d(in_channels, hidden_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(hidden_channels),
            nn.ReLU6(inplace=True),
            DepthwiseSeparableConv(hidden_channels, out_channels, stride=stride, expand_ratio=expand_ratio)
        )

        self.use_residual = stride == 1 and in_channels == out_channels

    def forward(self, x):
        if self.use_residual:
            return x + self.block(x)
        else:
            return self.block(x)

# MobileNetV2 模型
class MobileNetV2(nn.Module):
    def __init__(self, num_classes=1000):
        super(MobileNetV2, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU6(inplace=True),
            
            MobileNetV2Block(32, 16, stride=1, expand_ratio=1),
            MobileNetV2Block(16, 24, stride=2, expand_ratio=6),
            MobileNetV2Block(24, 24, stride=1, expand_ratio=6),
            MobileNetV2Block(24, 32, stride=2, expand_ratio=6),
            MobileNetV2Block(32, 32, stride=1, expand_ratio=6),
            MobileNetV2Block(32, 64, stride=2, expand_ratio=6),
            MobileNetV2Block(64, 64, stride=1, expand_ratio=6),
            MobileNetV2Block(64, 96, stride=1, expand_ratio=6),
            MobileNetV2Block(96, 96, stride=1, expand_ratio=6),
            MobileNetV2Block(96, 160, stride=2, expand_ratio=6),
            MobileNetV2Block(160, 160, stride=1, expand_ratio=6),
            MobileNetV2Block(160, 320, stride=1, expand_ratio=6),

            nn.Conv2d(320, 1280, kernel_size=1, bias=False),
            nn.BatchNorm2d(1280),
            nn.ReLU6(inplace=True),

            nn.AdaptiveAvgPool2d(1)
        )

        self.fc = nn.Linear(1280, num_classes)

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# 定义结合MobileNetV2特征和结构化特征的XGBoost模型
class CombinedModel(nn.Module):
    def __init__(self, mobilenet_model, structured_data_dim):
        super(CombinedModel, self).__init__()
        self.mobilenet = mobilenet_model
        self.structured_data_dim = structured_data_dim

    def forward(self, image, structured_data):
        image_features = self.mobilenet(image).detach().cpu().numpy()
        combined_features = np.hstack((image_features, structured_data.detach().cpu().numpy()))
        return combined_features

# 判断是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据变换
transform = transforms.Compose([
    transforms.Resize((15, 15)),  # 修改为15x15图像尺寸
    transforms.ToTensor(),
])

# 加载数据集
train_dataset = CustomDataset(
    csv_file='data/train_metadata_aligned.csv',
    img_hdf5_file='data/train-image.hdf5',
    transform=transform
)

# 划分训练集和验证集
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_set, val_set = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)

# 初始化模型
mobilenet_model = MobileNetV2(num_classes=1280).to(device)
combined_model = CombinedModel(mobilenet_model, structured_data_dim=train_set[0][1].shape[0])

# XGBoost分类器
xgb_model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100)

# 训练与验证
best_auc = 0
for epoch in range(20):  # 这里只用1个epoch演示，实际应使用更多
    train_features = []
    train_labels = []

    # 训练过程
    for images, structured_data, targets in train_loader:
        images, structured_data = images.to(device), structured_data.to(device)
        features = combined_model(images, structured_data)
        train_features.append(features)
        train_labels.append(targets.numpy())
    
    train_features = np.vstack(train_features)
    train_labels = np.hstack(train_labels)

    xgb_model.fit(train_features, train_labels)

    # 验证过程
    val_features = []
    val_labels = []

    for images, structured_data, targets in val_loader:
        images, structured_data = images.to(device), structured_data.to(device)
        features = combined_model(images, structured_data)
        val_features.append(features)
        val_labels.append(targets.numpy())

    val_features = np.vstack(val_features)
    val_labels = np.hstack(val_labels)
    
    val_preds = xgb_model.predict_proba(val_features)[:, 1]
    auc = roc_auc_score(val_labels, val_preds)
    print(f"Epoch {epoch+1}: AUC = {auc:.4f}")

    # 保存最佳模型
    if auc > best_auc:
        best_auc = auc
        torch.save(combined_model.state_dict(), '0906best_combined_model.pth')
        xgb_model.save_model('0906best_xgb_model.json')


# 测试过程
test_dataset = CustomDataset(
    csv_file='data/test-metadata.csv',
    img_hdf5_file='data/test-image.hdf5',
    transform=transform,
    is_test=True
)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 加载最佳模型
combined_model.load_state_dict(torch.load('0906best_combined_model.pth'))
xgb_model.load_model('0906best_xgb_model.json')

test_image_names = []
test_predictions = []

for images, structured_data, image_ids in test_loader:
    images, structured_data = images.to(device), structured_data.to(device)
    features = combined_model(images, structured_data)
    preds = xgb_model.predict_proba(features)[:, 1]
    test_image_names.extend(image_ids)  # 这里不需要调用 numpy()
    test_predictions.extend(preds)

# 保存测试结果
results = pd.DataFrame({'image_id': test_image_names, 'prediction': test_predictions})
results.to_csv('test_predictions.csv', index=False)

print("Testing completed. Predictions saved to test_predictions.csv")



KeyboardInterrupt: 

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import h5py
from PIL import Image
import io
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

# 数据集类
class CustomDataset(Dataset):
    def __init__(self, csv_file, img_hdf5_file, transform=None, is_test=False):
        self.data = pd.read_csv(csv_file)
        self.img_hdf5_file = img_hdf5_file
        self.transform = transform
        self.is_test = is_test
        
        # 处理图像ID
        self.image_ids = self.data['isic_id'].values
        
        # 处理结构化数据
        if 'target' in self.data.columns:
            self.structured_data = self.data.drop(columns=['isic_id', 'target'])
        else:
            self.structured_data = self.data.drop(columns=['isic_id'])
        
        # 特征工程：将字符串类型的列转换为数字
        for col in self.structured_data.columns:
            if self.structured_data[col].dtype == 'object':
                self.structured_data[col] = self.structured_data[col].astype(str).str.extract('(\d+)').astype(float)
        
        # 处理缺失值
        self.structured_data = self.structured_data.fillna(0).values.astype(np.float32)
        
        # 特征标准化
        self.scaler = StandardScaler()
        self.structured_data = self.scaler.fit_transform(self.structured_data)
        
        if not is_test:
            # 如果不是测试集，处理目标列
            self.targets = self.data['target'].values.astype(np.float32)
        else:
            self.targets = None

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        with h5py.File(self.img_hdf5_file, 'r') as hdf5_file:
            image_key = self.image_ids[idx]
            image_data = hdf5_file[image_key][()]
            
        image = Image.open(io.BytesIO(image_data))
        if self.transform:
            image = self.transform(image)
        
        structured_data = torch.tensor(self.structured_data[idx], dtype=torch.float32)
        if self.targets is not None:
            target = torch.tensor(self.targets[idx], dtype=torch.float32)
            return image, structured_data, target
        else:
            return image, structured_data, self.image_ids[idx]

# 深度可分离卷积（Depthwise Separable Convolution）
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, expand_ratio=1):
        super(DepthwiseSeparableConv, self).__init__()
        hidden_channels = in_channels * expand_ratio

        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, hidden_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False),
            nn.BatchNorm2d(hidden_channels),
            nn.ReLU6(inplace=True),
            nn.Conv2d(hidden_channels, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )

    def forward(self, x):
        return self.conv(x)

# MobileNetV2 的基本块（Block）
class MobileNetV2Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride, expand_ratio):
        super(MobileNetV2Block, self).__init__()
        hidden_channels = in_channels * expand_ratio

        self.block = nn.Sequential(
            nn.Conv2d(in_channels, hidden_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(hidden_channels),
            nn.ReLU6(inplace=True),
            DepthwiseSeparableConv(hidden_channels, out_channels, stride=stride, expand_ratio=expand_ratio)
        )

        self.use_residual = stride == 1 and in_channels == out_channels

    def forward(self, x):
        if self.use_residual:
            return x + self.block(x)
        else:
            return self.block(x)

# MobileNetV2 模型
class MobileNetV2(nn.Module):
    def __init__(self, num_classes=1000):
        super(MobileNetV2, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU6(inplace=True),
            
            MobileNetV2Block(32, 16, stride=1, expand_ratio=1),
            MobileNetV2Block(16, 24, stride=2, expand_ratio=6),
            MobileNetV2Block(24, 24, stride=1, expand_ratio=6),
            MobileNetV2Block(24, 32, stride=2, expand_ratio=6),
            MobileNetV2Block(32, 32, stride=1, expand_ratio=6),
            MobileNetV2Block(32, 64, stride=2, expand_ratio=6),
            MobileNetV2Block(64, 64, stride=1, expand_ratio=6),
            MobileNetV2Block(64, 96, stride=1, expand_ratio=6),
            MobileNetV2Block(96, 96, stride=1, expand_ratio=6),
            MobileNetV2Block(96, 160, stride=2, expand_ratio=6),
            MobileNetV2Block(160, 160, stride=1, expand_ratio=6),
            MobileNetV2Block(160, 320, stride=1, expand_ratio=6),

            nn.Conv2d(320, 1280, kernel_size=1, bias=False),
            nn.BatchNorm2d(1280),
            nn.ReLU6(inplace=True),

            nn.AdaptiveAvgPool2d(1)
        )

        self.fc = nn.Linear(1280, num_classes)

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# 定义结合MobileNetV2特征和结构化特征的XGBoost模型
class CombinedModel(nn.Module):
    def __init__(self, mobilenet_model, structured_data_dim):
        super(CombinedModel, self).__init__()
        self.mobilenet = mobilenet_model
        self.structured_data_dim = structured_data_dim

    def forward(self, image, structured_data):
        image_features = self.mobilenet(image).detach().cpu().numpy()
        combined_features = np.hstack((image_features, structured_data.detach().cpu().numpy()))
        return combined_features

# 判断是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据变换
transform = transforms.Compose([
    transforms.Resize((15, 15)),  # 修改为15x15图像尺寸
    transforms.ToTensor(),
])

# 加载数据集
train_dataset = CustomDataset(
    csv_file='data/train_metadata_aligned.csv',
    img_hdf5_file='data/train-image.hdf5',
    transform=transform
)

# 划分训练集和验证集
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_set, val_set = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)

# 初始化模型
mobilenet_model = MobileNetV2(num_classes=1280).to(device)
combined_model = CombinedModel(mobilenet_model, structured_data_dim=train_set[0][1].shape[0])

# XGBoost分类器
xgb_model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100)

# 测试过程
test_dataset = CustomDataset(
    csv_file='data/test-metadata.csv',
    img_hdf5_file='data/test-image.hdf5',
    transform=transform,
    is_test=True
)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 加载最佳模型
combined_model.load_state_dict(torch.load('0906best_combined_model.pth'))
xgb_model.load_model('0906best_xgb_model.json')

test_image_names = []
test_predictions = []

for images, structured_data, image_ids in test_loader:
    images, structured_data = images.to(device), structured_data.to(device)
    features = combined_model(images, structured_data)
    preds = xgb_model.predict_proba(features)[:, 1]
    test_image_names.extend(image_ids)  # 这里不需要调用 numpy()
    test_predictions.extend(preds)

# 保存测试结果
results = pd.DataFrame({'image_id': test_image_names, 'prediction': test_predictions})
results.to_csv('test_predictions.csv', index=False)

print("Testing completed. Predictions saved to test_predictions.csv")


Testing completed. Predictions saved to test_predictions.csv


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import h5py
from PIL import Image
import io
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

# 数据集类
class CustomDataset(Dataset):
    def __init__(self, csv_file, img_hdf5_file, transform=None, is_test=False):
        self.data = pd.read_csv(csv_file)
        self.img_hdf5_file = img_hdf5_file
        self.transform = transform
        self.is_test = is_test
        
        # 处理图像ID
        self.image_ids = self.data['isic_id'].values
        
        # 处理结构化数据
        if 'target' in self.data.columns:
            self.structured_data = self.data.drop(columns=['isic_id', 'target'])
        else:
            self.structured_data = self.data.drop(columns=['isic_id'])
        
        # 特征工程：将字符串类型的列转换为数字
        for col in self.structured_data.columns:
            if self.structured_data[col].dtype == 'object':
                self.structured_data[col] = self.structured_data[col].astype(str).str.extract('(\d+)').astype(float)
        
        # 处理缺失值
        self.structured_data = self.structured_data.fillna(0).values.astype(np.float32)
        
        # 特征标准化
        self.scaler = StandardScaler()
        self.structured_data = self.scaler.fit_transform(self.structured_data)
        
        if not is_test:
            # 如果不是测试集，处理目标列
            self.targets = self.data['target'].values.astype(np.float32)
        else:
            self.targets = None

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        with h5py.File(self.img_hdf5_file, 'r') as hdf5_file:
            image_key = self.image_ids[idx]
            image_data = hdf5_file[image_key][()]
            
        image = Image.open(io.BytesIO(image_data))
        if self.transform:
            image = self.transform(image)
        
        structured_data = torch.tensor(self.structured_data[idx], dtype=torch.float32)
        if self.targets is not None:
            target = torch.tensor(self.targets[idx], dtype=torch.float32)
            return image, structured_data, target
        else:
            return image, structured_data, self.image_ids[idx]

# 深度可分离卷积（Depthwise Separable Convolution）
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, expand_ratio=1):
        super(DepthwiseSeparableConv, self).__init__()
        hidden_channels = in_channels * expand_ratio

        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, hidden_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False),
            nn.BatchNorm2d(hidden_channels),
            nn.ReLU6(inplace=True),
            nn.Conv2d(hidden_channels, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )

    def forward(self, x):
        return self.conv(x)

# MobileNetV2 的基本块（Block）
class MobileNetV2Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride, expand_ratio):
        super(MobileNetV2Block, self).__init__()
        hidden_channels = in_channels * expand_ratio

        self.block = nn.Sequential(
            nn.Conv2d(in_channels, hidden_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(hidden_channels),
            nn.ReLU6(inplace=True),
            DepthwiseSeparableConv(hidden_channels, out_channels, stride=stride, expand_ratio=expand_ratio)
        )

        self.use_residual = stride == 1 and in_channels == out_channels

    def forward(self, x):
        if self.use_residual:
            return x + self.block(x)
        else:
            return self.block(x)

# MobileNetV2 模型
class MobileNetV2(nn.Module):
    def __init__(self, num_classes=1000):
        super(MobileNetV2, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU6(inplace=True),
            
            MobileNetV2Block(32, 16, stride=1, expand_ratio=1),
            MobileNetV2Block(16, 24, stride=2, expand_ratio=6),
            MobileNetV2Block(24, 24, stride=1, expand_ratio=6),
            MobileNetV2Block(24, 32, stride=2, expand_ratio=6),
            MobileNetV2Block(32, 32, stride=1, expand_ratio=6),
            MobileNetV2Block(32, 64, stride=2, expand_ratio=6),
            MobileNetV2Block(64, 64, stride=1, expand_ratio=6),
            MobileNetV2Block(64, 96, stride=1, expand_ratio=6),
            MobileNetV2Block(96, 96, stride=1, expand_ratio=6),
            MobileNetV2Block(96, 160, stride=2, expand_ratio=6),
            MobileNetV2Block(160, 160, stride=1, expand_ratio=6),
            MobileNetV2Block(160, 320, stride=1, expand_ratio=6),

            nn.Conv2d(320, 1280, kernel_size=1, bias=False),
            nn.BatchNorm2d(1280),
            nn.ReLU6(inplace=True),

            nn.AdaptiveAvgPool2d(1)
        )

        self.fc = nn.Linear(1280, num_classes)

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# 定义结合MobileNetV2特征和结构化特征的XGBoost模型
class CombinedModel(nn.Module):
    def __init__(self, mobilenet_model, structured_data_dim):
        super(CombinedModel, self).__init__()
        self.mobilenet = mobilenet_model
        self.structured_data_dim = structured_data_dim

    def forward(self, image, structured_data):
        image_features = self.mobilenet(image).detach().cpu().numpy()
        combined_features = np.hstack((image_features, structured_data.detach().cpu().numpy()))
        return combined_features

# 判断是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据变换
transform = transforms.Compose([
    transforms.Resize((15, 15)),  # 修改为15x15图像尺寸
    transforms.ToTensor(),
])

# 加载数据集
train_dataset = CustomDataset(
    csv_file='data/train_metadata_aligned.csv',
    img_hdf5_file='data/train-image.hdf5',
    transform=transform
)

# 划分训练集和验证集
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_set, val_set = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)

# 初始化模型
mobilenet_model = MobileNetV2(num_classes=1280).to(device)
combined_model = CombinedModel(mobilenet_model, structured_data_dim=train_set[0][1].shape[0])

# XGBoost分类器
xgb_model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100)

# 测试过程
test_dataset = CustomDataset(
    csv_file='data/test-metadata.csv',
    img_hdf5_file='data/test-image.hdf5',
    transform=transform,
    is_test=True
)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 加载最佳模型
combined_model.load_state_dict(torch.load('0906best_combined_model.pth'))
xgb_model.load_model('0906best_xgb_model.json')

test_image_names = []
test_predictions = []

for images, structured_data, image_ids in test_loader:
    images, structured_data = images.to(device), structured_data.to(device)
    features = combined_model(images, structured_data)
    preds = xgb_model.predict_proba(features)[:, 1]
    test_image_names.extend(image_ids)  # 这里不需要调用 numpy()
    test_predictions.extend(preds)

# 保存测试结果
results = pd.DataFrame({'image_id': test_image_names, 'prediction': test_predictions})
results.to_csv('test_predictions.csv', index=False)

print("Testing completed. Predictions saved to test_predictions.csv")


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import h5py
from PIL import Image
import io
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

# 数据集类
class CustomDataset(Dataset):
    def __init__(self, csv_file, img_hdf5_file, transform=None, is_test=False):
        self.data = pd.read_csv(csv_file)
        self.img_hdf5_file = img_hdf5_file
        self.transform = transform
        self.is_test = is_test
        
        # 处理图像ID
        self.image_ids = self.data['isic_id'].values
        
        # 处理结构化数据
        if 'target' in self.data.columns:
            self.structured_data = self.data.drop(columns=['isic_id', 'target'])
        else:
            self.structured_data = self.data.drop(columns=['isic_id'])
        
        # 特征工程：将字符串类型的列转换为数字
        for col in self.structured_data.columns:
            if self.structured_data[col].dtype == 'object':
                self.structured_data[col] = self.structured_data[col].astype(str).str.extract('(\d+)').astype(float)
        
        # 处理缺失值
        self.structured_data = self.structured_data.fillna(0).values.astype(np.float32)
        
        # 特征标准化
        self.scaler = StandardScaler()
        self.structured_data = self.scaler.fit_transform(self.structured_data)
        
        if not is_test:
            # 如果不是测试集，处理目标列
            self.targets = self.data['target'].values.astype(np.float32)
        else:
            self.targets = None

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        with h5py.File(self.img_hdf5_file, 'r') as hdf5_file:
            image_key = self.image_ids[idx]
            image_data = hdf5_file[image_key][()]
            
        image = Image.open(io.BytesIO(image_data))
        if self.transform:
            image = self.transform(image)
        
        structured_data = torch.tensor(self.structured_data[idx], dtype=torch.float32)
        if self.targets is not None:
            target = torch.tensor(self.targets[idx], dtype=torch.float32)
            return image, structured_data, target
        else:
            return image, structured_data, self.image_ids[idx]

# 定义SimpleCNN模型
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=1280):  # 输出1280与之前MobileNetV2保持一致
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc = nn.Linear(64 * 1 * 1, num_classes)  # 假设最终特征图大小为1x1
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)  # 展平
        x = self.fc(x)
        return x

# 定义结合SimpleCNN特征和结构化特征的XGBoost模型
class CombinedModel(nn.Module):
    def __init__(self, simplecnn_model, structured_data_dim):
        super(CombinedModel, self).__init__()
        self.simplecnn = simplecnn_model
        self.structured_data_dim = structured_data_dim

    def forward(self, image, structured_data):
        image_features = self.simplecnn(image).detach().cpu().numpy()
        combined_features = np.hstack((image_features, structured_data.detach().cpu().numpy()))
        return combined_features


# 判断是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据变换
transform = transforms.Compose([
    transforms.Resize((15, 15)),  # 修改为15x15图像尺寸
    transforms.ToTensor(),
])

# 加载数据集
train_dataset = CustomDataset(
    csv_file='data/train_metadata_aligned.csv',
    img_hdf5_file='data/train-image.hdf5',
    transform=transform
)

# 划分训练集和验证集
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_set, val_set = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)

# 初始化模型
mobilenet_model = MobileNetV2(num_classes=1280).to(device)
combined_model = CombinedModel(mobilenet_model, structured_data_dim=train_set[0][1].shape[0])

# XGBoost分类器
xgb_model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100)

# 训练与验证
best_auc = 0
for epoch in range(20):  # 这里只用1个epoch演示，实际应使用更多
    train_features = []
    train_labels = []

    # 训练过程
    for images, structured_data, targets in train_loader:
        images, structured_data = images.to(device), structured_data.to(device)
        features = combined_model(images, structured_data)
        train_features.append(features)
        train_labels.append(targets.numpy())
    
    train_features = np.vstack(train_features)
    train_labels = np.hstack(train_labels)

    xgb_model.fit(train_features, train_labels)

    # 验证过程
    val_features = []
    val_labels = []

    for images, structured_data, targets in val_loader:
        images, structured_data = images.to(device), structured_data.to(device)
        features = combined_model(images, structured_data)
        val_features.append(features)
        val_labels.append(targets.numpy())

    val_features = np.vstack(val_features)
    val_labels = np.hstack(val_labels)
    
    val_preds = xgb_model.predict_proba(val_features)[:, 1]
    auc = roc_auc_score(val_labels, val_preds)
    print(f"Epoch {epoch+1}: AUC = {auc:.4f}")

    # 保存最佳模型
    if auc > best_auc:
        best_auc = auc
        torch.save(combined_model.state_dict(), '0907best_combined_model.pth')
        xgb_model.save_model('0907best_xgb_model.json')


# 测试过程
test_dataset = CustomDataset(
    csv_file='data/test-metadata.csv',
    img_hdf5_file='data/test-image.hdf5',
    transform=transform,
    is_test=True
)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 加载最佳模型
combined_model.load_state_dict(torch.load('0907best_combined_model.pth'))
xgb_model.load_model('0907best_xgb_model.json')

test_image_names = []
test_predictions = []

for images, structured_data, image_ids in test_loader:
    images, structured_data = images.to(device), structured_data.to(device)
    features = combined_model(images, structured_data)
    preds = xgb_model.predict_proba(features)[:, 1]
    test_image_names.extend(image_ids)  # 这里不需要调用 numpy()
    test_predictions.extend(preds)

# 保存测试结果
results = pd.DataFrame({'image_id': test_image_names, 'prediction': test_predictions})
results.to_csv('test_predictions.csv', index=False)

print("Testing completed. Predictions saved to test_predictions.csv")




Epoch 1: AUC = 0.9169
Epoch 2: AUC = 0.9343
Epoch 3: AUC = 0.9232
Epoch 4: AUC = 0.9235
Epoch 5: AUC = 0.9044
Epoch 6: AUC = 0.9291
Epoch 7: AUC = 0.9349
Epoch 8: AUC = 0.9116
Epoch 9: AUC = 0.9090
Epoch 10: AUC = 0.9216


KeyboardInterrupt: 

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import h5py
from PIL import Image
import io
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

# 数据集类
class CustomDataset(Dataset):
    def __init__(self, csv_file, img_hdf5_file, transform=None, is_test=False):
        self.data = pd.read_csv(csv_file)
        self.img_hdf5_file = img_hdf5_file
        self.transform = transform
        self.is_test = is_test
        
        # 处理图像ID
        self.image_ids = self.data['isic_id'].values
        
        # 处理结构化数据
        if 'target' in self.data.columns:
            self.structured_data = self.data.drop(columns=['isic_id', 'target'])
        else:
            self.structured_data = self.data.drop(columns=['isic_id'])
        
        # 特征工程：将字符串类型的列转换为数字
        for col in self.structured_data.columns:
            if self.structured_data[col].dtype == 'object':
                self.structured_data[col] = self.structured_data[col].astype(str).str.extract('(\d+)').astype(float)
        
        # 处理缺失值
        self.structured_data = self.structured_data.fillna(0).values.astype(np.float32)
        
        # 特征标准化
        self.scaler = StandardScaler()
        self.structured_data = self.scaler.fit_transform(self.structured_data)
        
        if not is_test:
            # 如果不是测试集，处理目标列
            self.targets = self.data['target'].values.astype(np.float32)
        else:
            self.targets = None

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        with h5py.File(self.img_hdf5_file, 'r') as hdf5_file:
            image_key = self.image_ids[idx]
            image_data = hdf5_file[image_key][()]
            
        image = Image.open(io.BytesIO(image_data))
        if self.transform:
            image = self.transform(image)
        
        structured_data = torch.tensor(self.structured_data[idx], dtype=torch.float32)
        if self.targets is not None:
            target = torch.tensor(self.targets[idx], dtype=torch.float32)
            return image, structured_data, target
        else:
            return image, structured_data, self.image_ids[idx]

# 深度可分离卷积（Depthwise Separable Convolution）
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, expand_ratio=1):
        super(DepthwiseSeparableConv, self).__init__()
        hidden_channels = in_channels * expand_ratio

        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, hidden_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False),
            nn.BatchNorm2d(hidden_channels),
            nn.ReLU6(inplace=True),
            nn.Conv2d(hidden_channels, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )

    def forward(self, x):
        return self.conv(x)

# MobileNetV2 的基本块（Block）
class MobileNetV2Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride, expand_ratio):
        super(MobileNetV2Block, self).__init__()
        hidden_channels = in_channels * expand_ratio

        self.block = nn.Sequential(
            nn.Conv2d(in_channels, hidden_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(hidden_channels),
            nn.ReLU6(inplace=True),
            DepthwiseSeparableConv(hidden_channels, out_channels, stride=stride, expand_ratio=expand_ratio)
        )

        self.use_residual = stride == 1 and in_channels == out_channels

    def forward(self, x):
        if self.use_residual:
            return x + self.block(x)
        else:
            return self.block(x)

# MobileNetV2 模型
class MobileNetV2(nn.Module):
    def __init__(self, num_classes=1000):
        super(MobileNetV2, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU6(inplace=True),
            
            MobileNetV2Block(32, 16, stride=1, expand_ratio=1),
            MobileNetV2Block(16, 24, stride=2, expand_ratio=6),
            MobileNetV2Block(24, 24, stride=1, expand_ratio=6),
            MobileNetV2Block(24, 32, stride=2, expand_ratio=6),
            MobileNetV2Block(32, 32, stride=1, expand_ratio=6),
            MobileNetV2Block(32, 64, stride=2, expand_ratio=6),
            MobileNetV2Block(64, 64, stride=1, expand_ratio=6),
            MobileNetV2Block(64, 96, stride=1, expand_ratio=6),
            MobileNetV2Block(96, 96, stride=1, expand_ratio=6),
            MobileNetV2Block(96, 160, stride=2, expand_ratio=6),
            MobileNetV2Block(160, 160, stride=1, expand_ratio=6),
            MobileNetV2Block(160, 320, stride=1, expand_ratio=6),

            nn.Conv2d(320, 1280, kernel_size=1, bias=False),
            nn.BatchNorm2d(1280),
            nn.ReLU6(inplace=True),

            nn.AdaptiveAvgPool2d(1)
        )

        self.fc = nn.Linear(1280, num_classes)

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# 定义结合MobileNetV2特征和结构化特征的XGBoost模型
class CombinedModel(nn.Module):
    def __init__(self, mobilenet_model, structured_data_dim):
        super(CombinedModel, self).__init__()
        self.mobilenet = mobilenet_model
        self.structured_data_dim = structured_data_dim

    def forward(self, image, structured_data):
        image_features = self.mobilenet(image).detach().cpu().numpy()
        combined_features = np.hstack((image_features, structured_data.detach().cpu().numpy()))
        return combined_features

# 判断是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据变换
transform = transforms.Compose([
    transforms.Resize((15, 15)),  # 修改为15x15图像尺寸
    transforms.ToTensor(),
])

# 加载数据集
train_dataset = CustomDataset(
    csv_file='data/train_metadata_aligned.csv',
    img_hdf5_file='data/train-image.hdf5',
    transform=transform
)

# 划分训练集和验证集
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_set, val_set = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)

# 初始化模型
mobilenet_model = MobileNetV2(num_classes=1280).to(device)
combined_model = CombinedModel(mobilenet_model, structured_data_dim=train_set[0][1].shape[0])

# XGBoost分类器
xgb_model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100)

# 训练与验证
best_auc = 0
for epoch in range(20):  # 这里只用1个epoch演示，实际应使用更多
    train_features = []
    train_labels = []

    # 训练过程
    for images, structured_data, targets in train_loader:
        images, structured_data = images.to(device), structured_data.to(device)
        features = combined_model(images, structured_data)
        train_features.append(features)
        train_labels.append(targets.numpy())
    
    train_features = np.vstack(train_features)
    train_labels = np.hstack(train_labels)

    xgb_model.fit(train_features, train_labels)

    # 验证过程
    val_features = []
    val_labels = []

    for images, structured_data, targets in val_loader:
        images, structured_data = images.to(device), structured_data.to(device)
        features = combined_model(images, structured_data)
        val_features.append(features)
        val_labels.append(targets.numpy())

    val_features = np.vstack(val_features)
    val_labels = np.hstack(val_labels)
    
    val_preds = xgb_model.predict_proba(val_features)[:, 1]
    auc = roc_auc_score(val_labels, val_preds)
    print(f"Epoch {epoch+1}: AUC = {auc:.4f}")

    # 保存最佳模型
    if auc > best_auc:
        best_auc = auc
        torch.save(combined_model.state_dict(), '0906best_combined_model.pth')
        xgb_model.save_model('0906best_xgb_model.json')


# 测试过程
test_dataset = CustomDataset(
    csv_file='data/test-metadata.csv',
    img_hdf5_file='data/test-image.hdf5',
    transform=transform,
    is_test=True
)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 加载最佳模型
combined_model.load_state_dict(torch.load('0906best_combined_model.pth'))
xgb_model.load_model('0906best_xgb_model.json')

test_image_names = []
test_predictions = []

for images, structured_data, image_ids in test_loader:
    images, structured_data = images.to(device), structured_data.to(device)
    features = combined_model(images, structured_data)
    preds = xgb_model.predict_proba(features)[:, 1]
    test_image_names.extend(image_ids)  # 这里不需要调用 numpy()
    test_predictions.extend(preds)

# 保存测试结果
results = pd.DataFrame({'image_id': test_image_names, 'prediction': test_predictions})
results.to_csv('test_predictions.csv', index=False)

print("Testing completed. Predictions saved to test_predictions.csv")



Epoch 1: AUC = 0.8853
Epoch 2: AUC = 0.9002
Epoch 3: AUC = 0.8915
Epoch 4: AUC = 0.8891
Epoch 5: AUC = 0.9104
Epoch 6: AUC = 0.8738
Epoch 7: AUC = 0.9025
Epoch 8: AUC = 0.8645
Epoch 9: AUC = 0.8632
Epoch 10: AUC = 0.9111
Epoch 11: AUC = 0.8872
Epoch 12: AUC = 0.9145
Epoch 13: AUC = 0.9175
Epoch 14: AUC = 0.8934
Epoch 15: AUC = 0.8832
Epoch 16: AUC = 0.8884
Epoch 17: AUC = 0.8911
Epoch 18: AUC = 0.8998
Epoch 19: AUC = 0.9208
Epoch 20: AUC = 0.8829
Testing completed. Predictions saved to test_predictions.csv
